In [2]:
import pandas as pd
import numpy as np
import os

In [3]:
chapter1 = pd.read_csv("C:/Users/mynam/TariffBot/data/section1_data/htsdata_Chapter1.csv")

In [4]:
chapter1.shape

(126, 9)

In [5]:
chapter1.sample(10)

,HTS Number,Indent,Description,Unit of Quantity,General Rate of Duty,Special Rate of Duty,Column 2 Rate of Duty,Quota Quantity,Additional Duties
89,NaN,3,"Breeding stock, whether or not purebred:",NaN,NaN,NaN,NaN,NaN,NaN
20,0102.21.00.50,4,Female,"[""No.""]",NaN,NaN,NaN,NaN,NaN
55,0102.39.00.24,5,Male,"[""No."",""kg""]",NaN,NaN,NaN,NaN,NaN
88,0105.11.00,2,Chickens,NaN,0.9¢ each,"Free (A+,AU,BH,CL,CO,D,E, IL,JO,KR, MA,OM, P,P...",4¢ each,NaN,NaN
8,0101.30.00.00,1,Asses,"[""No.""]",6.8%,"Free (A+,AU,BH,CL,CO,D,E,IL,JO,KR,MA,OM,P,PA,P...",15%,NaN,NaN
6,0101.29.00.10,3,Imported for immediate slaughter,"[""No.""]",NaN,NaN,NaN,NaN,NaN
108,0106.19.91,3,Other,NaN,Free,NaN,15%,NaN,NaN
10,0101.90.30.00,2,Imported for immediate slaughter,"[""No.""]",Free,NaN,Free,NaN,NaN
97,NaN,1,Other:,NaN,NaN,NaN,NaN,NaN,NaN
94,0105.13.00.00,2,Ducks,"[""No.""]",0.9¢ each,"Free (A+,AU,BH,CL,CO,D,E, IL,JO,KR,MA,OM,P,PA,...",4¢ each,NaN,NaN


In [6]:
chapter2 = pd.read_csv("C:\\Users\\mynam\\TariffBot\\data\\section1_data\\htsdata_Chapter2.csv")

In [7]:
chapter3 = pd.read_csv("C:\\Users\\mynam\\TariffBot\\data\\section1_data\\htsdata_Chapter3.csv")
chapter4 = pd.read_csv("C:\\Users\\mynam\\TariffBot\\data\\section1_data\\htsdata_Chapter4.csv")
chapter5 = pd.read_csv("C:\\Users\\mynam\\TariffBot\\data\\section1_data\\htsdata_Chapter5.csv")

In [8]:
chapter2.head()

,HTS Number,Indent,Description,Unit of Quantity,General Rate of Duty,Special Rate of Duty,Column 2 Rate of Duty,Quota Quantity,Additional Duties
0,0201,0,"Meat of bovine animals, fresh or chilled:",NaN,NaN,NaN,NaN,NaN,NaN
1,0201.10,1,Carcasses and half-carcasses:,NaN,NaN,NaN,NaN,NaN,NaN
2,0201.10.05,2,Described in general note 15 of the tariff sch...,NaN,4.4¢/kg,"Free (A+,AU,BH,CL,CO,D,E*, IL,JO,KR,MA,OM,P,PA...",13.2¢/kg,NaN,NaN
3,0201.10.05.10,3,Veal,"[""kg""]",NaN,NaN,NaN,NaN,NaN
4,0201.10.05.90,3,Other,"[""kg""]",NaN,NaN,NaN,NaN,NaN


In [9]:
chapter3.sample(6)

,HTS Number,Indent,Description,Unit of Quantity,General Rate of Duty,Special Rate of Duty,Column 2 Rate of Duty,Quota Quantity,Additional Duties
456,0304.93,2,"Tilapias (Oreochromis spp.), catfish (Pangasiu...",NaN,NaN,NaN,NaN,NaN,NaN
649,0306.19.01.30,3,Antarctic krill (Euphausia superba),"[""kg""]",NaN,NaN,NaN,NaN,NaN
332,0304.53.00.20,3,Haddock,"[""kg""]",NaN,NaN,NaN,NaN,NaN
576,0305.69.30.00,4,Other,"[""kg""]",Free,NaN,2.2¢/kg,NaN,NaN
496,NaN,5,Other:,NaN,NaN,NaN,NaN,NaN,NaN
366,0304.62.00.90,3,Other,"[""kg""]",NaN,NaN,NaN,NaN,NaN


In [10]:
chapter4.head()

,HTS Number,Indent,Description,Unit of Quantity,General Rate of Duty,Special Rate of Duty,Column 2 Rate of Duty,Quota Quantity,Additional Duties
0,0401,0,"Milk and cream, not concentrated nor containin...",NaN,NaN,NaN,NaN,NaN,NaN
1,0401.10.00.00,1,"Of a fat content, by weight, not exceeding 1 p...","[""liters"",""kg""]",0.34¢/liter,"Free (A+,AU,BH,CL,CO,D,E,IL,JO,KR,MA,OM,P,PA,P...",0.5¢/liter,NaN,NaN
2,0401.20,1,"Of a fat content, by weight, exceeding 1 perce...",NaN,NaN,NaN,NaN,NaN,NaN
3,0401.20.20.00,2,"For not over 11,356,236 liters entered in any ...","[""liters"",""kg cmsc""]",0.43¢/liter,"Free (A+,AU,BH,CL,CO,D,E,IL,JO,KR,MA,OM,P,PA,P...",1.7¢/liter,NaN,NaN
4,0401.20.40.00,2,Other,"[""liters"",""kg cmsc""]",1.5¢/liter,"Free (AU,BH,CL,CO,IL,JO,KR,MA,OM,P,PA,PE,S,SG)",1.7¢/liter,NaN,NaN


In [11]:
chapter5.head()

,HTS Number,Indent,Description,Unit of Quantity,General Rate of Duty,Special Rate of Duty,Column 2 Rate of Duty,Quota Quantity,Additional Duties
0,0501.00.00.00,0,"Human hair, unworked, whether or not washed or...","[""kg""]",1.4%,"Free (A,AU,BH,CL,CO,D,E,IL,JO,KR,MA,OM,P,PA,PE...",20%,NaN,NaN
1,0502,0,"Pigs', hogs' or boars' bristles and hair; badg...",NaN,NaN,NaN,NaN,NaN,NaN
2,0502.10.00.00,1,"Pigs', hogs' or boars' bristles and hair and w...","[""kg""]",0.8¢/kg,"Free (A*,AU,BH,CL,CO,D,E,IL,JO,KR,MA,OM,P,PA,P...",6.6¢/kg,NaN,NaN
3,0502.90.00.00,1,Other,"[""kg""]",Free,NaN,Free,NaN,NaN
4,0504.00.00,0,"Guts, bladders and stomachs of animals (other ...",NaN,Free,NaN,Free,NaN,NaN


In [12]:
merged_df = pd.concat([chapter1, chapter2, chapter3, chapter4, chapter5], ignore_index=True)

In [14]:
merged_df.columns

Index(['HTS Number', 'Indent', 'Description', 'Unit of Quantity',
       'General Rate of Duty', 'Special Rate of Duty', 'Column 2 Rate of Duty',
       'Quota Quantity', 'Additional Duties'],
      dtype='object')

In [15]:
merged_df.to_csv("section1.csv",index=False)

In [ ]:
import re

# Load HTS data CSV
df = pd.read_csv("htsdata.csv")

# Input values
product_cost = 10000.0  # FOB cost
freight = 500.0
insurance = 100.0
unit_weight = 1000  # in kg
quantity = 10       # number of units

# Compute CIF
cif_value = product_cost + freight + insurance

# Duty parser
def parse_duty_advanced(duty_str, unit_weight=None, quantity=None):
    """
    Parses and returns a duty rate as a decimal fraction (e.g., 0.05 for 5%),
    or a calculated specific duty based on unit_weight or quantity.
    """
    if pd.isna(duty_str) or duty_str.strip() == "":
        return 0.0

    duty_str = duty_str.strip().lower()

    if "free" in duty_str:
        return 0.0

    # Percentage duty (e.g., '5%')
    match = re.search(r"([\d.]+)\s*%", duty_str)
    if match:
        return float(match.group(1)) / 100

    # Weight-based duty (e.g., '2.5¢/kg')
    match = re.search(r"([\d.]+)\s*¢/kg", duty_str)
    if match and unit_weight is not None:
        cents_per_kg = float(match.group(1))
        return (cents_per_kg * unit_weight) / (100 * cif_value)  # convert to % of CIF

    # Unit-based duty (e.g., '$1.00/unit')
    match = re.search(r"\$([\d.]+)/unit", duty_str)
    if match and quantity is not None:
        dollars_per_unit = float(match.group(1))
        return (dollars_per_unit * quantity) / cif_value

    return 0.0

# Create working DataFrame
duty_df = df[["HTS Number", "Description", "General Rate of Duty", 
              "Special Rate of Duty", "Column 2 Rate of Duty"]].copy()

# Apply CIF and input data
duty_df["CIF Value"] = cif_value
duty_df["Product Cost"] = product_cost
duty_df["Freight"] = freight
duty_df["Insurance"] = insurance

# Apply duty calculations
for col in ["General Rate of Duty", "Special Rate of Duty", "Column 2 Rate of Duty"]:
    parsed_col = f"{col} Parsed (%)"
    amount_col = f"{col} Duty Amount"
    duty_df[parsed_col] = duty_df[col].apply(lambda x: parse_duty_advanced(x, unit_weight, quantity))
    duty_df[amount_col] = duty_df[parsed_col] * cif_value

# Filter results with any applicable duty
duty_df_filtered = duty_df[
    (duty_df["General Rate of Duty Duty Amount"] > 0) |
    (duty_df["Special Rate of Duty Duty Amount"] > 0) |
    (duty_df["Column 2 Rate of Duty Duty Amount"] > 0)
]

# Save results
duty_df_filtered.to_csv("landed_cost_duties.csv", index=False)
